[Reference](https://blog.stackademic.com/graphql-with-fastapi-866d52f1b32a)

# 1. Setting Up Your Environment

In [1]:
pip install fastapi uvicorn graphene

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.0 MB/s eta 0:00:00


# 2. Creating the Project Structure

```
my_project/
├── app/
│   ├── __init__.py
│   ├── main.py
│   ├── models.py
│   ├── crud.py
│   ├── schemas.py
│   └── database.py
├── requirements.txt
└── README.md
```

# 3. Configuring the Database (Optional)
For this example, let’s assume you are using a simple in-memory data store for simplicity. However, you can easily extend this to use any database of your choice.

# 4. Creating GraphQL Schemas with Graphene

In [2]:
# schemas.py
import graphene

class Item(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    description = graphene.String()

class Query(graphene.ObjectType):
    items = graphene.List(Item)

    def resolve_items(self, info):
        # Normally, this data would come from a database
        return [
            Item(id=1, name="Item 1", description="Description 1"),
            Item(id=2, name="Item 2", description="Description 2")
        ]

class CreateItem(graphene.Mutation):
    class Arguments:
        name = graphene.String(required=True)
        description = graphene.String(required=True)

    item = graphene.Field(lambda: Item)

    def mutate(self, info, name, description):
        item = Item(name=name, description=description)
        # Here you would save the item to the database
        return CreateItem(item=item)

class Mutation(graphene.ObjectType):
    create_item = CreateItem.Field()

schema = graphene.Schema(query=Query, mutation=Mutation)

# 5. Creating CRUD Operations

In [4]:
# crud.py
from typing import List
from .schemas import Item

items = []

def get_items() -> List[Item]:
    return items

def create_item(name: str, description: str) -> Item:
    item = Item(id=len(items) + 1, name=name, description=description)
    items.append(item)
    return item

# 6. Creating API Endpoints

In [5]:
# main.py
from fastapi import FastAPI
from starlette.graphql import GraphQLApp
from .schemas import schema

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

app.add_route("/graphql", GraphQLApp(schema=schema))

# 7. Running the Application

```
uvicorn app.main:app --reload
```